In [4]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score
from sklearn.model_selection import GridSearchCV
import matplotlib.pyplot as plt
import seaborn as sns

# Load the dataset
df = pd.read_csv('client_portfolio_data.csv')

# Examine basic information
print(f"Dataset shape: {df.shape}")
print(df['risk_appetite_label'].value_counts())

# Data preprocessing
# Identify categorical and numerical columns
categorical_cols = ['client_id', 'income_bracket', 'employment_status', 'education_level', 'holdings']
numerical_cols = [col for col in df.columns if col not in categorical_cols + ['risk_appetite_label']]

# Handle missing values
for col in numerical_cols:
    df[col].fillna(df[col].median(), inplace=True)
    
for col in categorical_cols:
    df[col].fillna(df[col].mode()[0], inplace=True)

# Drop client_id as it's an identifier
if 'client_id' in df.columns:
    categorical_cols.remove('client_id')
    df = df.drop('client_id', axis=1)

# Define preprocessing steps
preprocessor = ColumnTransformer(
    transformers=[
        ('num', StandardScaler(), numerical_cols),
        ('cat', OneHotEncoder(handle_unknown='ignore'), categorical_cols)
    ])

# Define features and target
X = df.drop('risk_appetite_label', axis=1)
y = df['risk_appetite_label']

# Split the data into training, validation and testing sets
X_train, X_temp, y_train, y_temp = train_test_split(X, y, test_size=0.3, random_state=42, stratify=y)
X_val, X_test, y_val, y_test = train_test_split(X_temp, y_temp, test_size=0.5, random_state=42, stratify=y_temp)

print(f"Training set size: {X_train.shape[0]}")
print(f"Validation set size: {X_val.shape[0]}")
print(f"Test set size: {X_test.shape[0]}")

# Create a pipeline with preprocessing and model
pipeline = Pipeline(steps=[
    ('preprocessor', preprocessor),
    ('classifier', GradientBoostingClassifier(random_state=42))
])

# Define hyperparameters for tuning
param_grid = {
            'classifier__n_estimators': [100, 200, 300],
            'classifier__learning_rate': [0.01, 0.05, 0.1],
            'classifier__max_depth': [3, 5, 7]
        }

# GridSearch with cross-validation
grid_search = GridSearchCV(pipeline, param_grid, cv=5, scoring='accuracy')
grid_search.fit(X_train, y_train)

# Best model
best_model = grid_search.best_estimator_

# Evaluation on validation set
y_val_pred = best_model.predict(X_val)
print("\nValidation Set Performance:")
print(f"Accuracy: {accuracy_score(y_val, y_val_pred):.4f}")
print("\nClassification Report:")
print(classification_report(y_val, y_val_pred))

# Confusion Matrix for validation set
plt.figure(figsize=(10, 8))
conf_matrix = confusion_matrix(y_val, y_val_pred)
sns.heatmap(conf_matrix, annot=True, fmt='d', cmap='Blues', 
            xticklabels=best_model.classes_, 
            yticklabels=best_model.classes_)
plt.xlabel('Predicted')
plt.ylabel('Actual')
plt.title('Confusion Matrix (Validation Set)')
plt.savefig('confusion_matrix_validation.png')
plt.close()

# Evaluation on test set
y_test_pred = best_model.predict(X_test)
print("\nTest Set Performance:")
print(f"Accuracy: {accuracy_score(y_test, y_test_pred):.4f}")
print("\nClassification Report:")
print(classification_report(y_test, y_test_pred))





Dataset shape: (2000, 29)
risk_appetite_label
Moderate                   622
Moderately Conservative    521
Conservative               385
Moderately Aggressive      298
Aggressive                 174
Name: count, dtype: int64
Training set size: 1400
Validation set size: 300
Test set size: 300


/tmp/ipykernel_27052/3420374621.py:27: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df[col].fillna(df[col].median(), inplace=True)
/tmp/ipykernel_27052/3420374621.py:30: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', tr


Validation Set Performance:
Accuracy: 0.9667

Classification Report:
                         precision    recall  f1-score   support

             Aggressive       0.89      0.92      0.91        26
           Conservative       0.98      0.98      0.98        58
               Moderate       0.97      1.00      0.98        93
  Moderately Aggressive       0.95      0.87      0.91        45
Moderately Conservative       0.99      0.99      0.99        78

               accuracy                           0.97       300
              macro avg       0.96      0.95      0.95       300
           weighted avg       0.97      0.97      0.97       300


Test Set Performance:
Accuracy: 0.9433

Classification Report:
                         precision    recall  f1-score   support

             Aggressive       0.92      0.88      0.90        26
           Conservative       0.98      0.95      0.96        58
               Moderate       0.93      0.99      0.96        94
  Moderately Aggr

In [6]:
# Confusion Matrix for test set
plt.figure(figsize=(10, 8))
conf_matrix = confusion_matrix(y_test, y_test_pred)
sns.heatmap(conf_matrix, annot=True, fmt='d', cmap='Blues', 
            xticklabels=best_model.classes_, 
            yticklabels=best_model.classes_)
plt.xlabel('Predicted')
plt.ylabel('Actual')
plt.title('Confusion Matrix (Test Set)')
plt.savefig('confusion_matrix_test.png')
plt.close()

# Feature importance (for the Random Forest component)
if hasattr(best_model.named_steps['classifier'], 'feature_importances_'):
    # Get feature names after preprocessing
    feature_names = []
    for name, trans, cols in preprocessor.transformers:
        if name == 'cat':
            # Get one-hot encoded feature names for categorical variables
            for i, col in enumerate(cols):
                categories = trans.categories[i]
                for cat in categories:
                    feature_names.append(f"{col}_{cat}")
        else:
            # Add numerical feature names as is
            feature_names.extend(cols)
    
    # Extract feature importances
    importances = best_model.named_steps['classifier'].feature_importances_
    
    # Sort feature importances in descending order
    indices = np.argsort(importances)[::-1]
    
    # Plot feature importances
    plt.figure(figsize=(12, 8))
    plt.title('Feature Importances for Risk Appetite Prediction')
    plt.bar(range(len(indices)), importances[indices], align='center')
    # plt.xticks(range(len(indices)), [feature_names[i] for i in indices], rotation=90)
    plt.tight_layout()
    plt.savefig('feature_importances.png')
    plt.close()
    
    # Print top 15 features
    # print("\nTop 15 important features:")
    # for i in indices[:15]:
    #     print(f"{feature_names[i]}: {importances[i]:.4f}")

# Save the model
import joblib
joblib.dump(best_model, 'Gradient_Boosting_risk_appetite_model.pkl')
print("\nModel saved as 'Gradient_Boosting_risk_appetite_model.pkl'")


Model saved as 'Gradient_Boosting_risk_appetite_model.pkl'


In [7]:
print("\nExample of using the prediction function:")
import pandas as pd
import numpy as np
import pickle
import joblib
with open('Gradient_Boosting_risk_appetite_model.pkl', 'rb') as file:
    loaded_model =  joblib.load(file)
df = pd.read_csv('client_portfolio_data.csv')
last = df.drop('risk_appetite_label', axis=1).tail(1)
predictions = loaded_model.predict(last)
print("Prediction")
print(predictions)


Example of using the prediction function:
Prediction
['Conservative']
